In [3]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2022, day=7)

def parses(input):
    commands = [grp for grp in input.strip().split('$ ')]
    
    hierarchy = {'/': {}}
    pwd = hierarchy
    
    stack = []
    for command in commands:
        if command == '':
            continue
        cmd, *output = command.strip().split('\n')
        if cmd == 'cd ..':
            pwd = stack.pop()
        elif cmd.startswith('cd'):
            _, arg = cmd.split(' ')
            stack.append(pwd)
            pwd = pwd.setdefault(arg, {})
        elif cmd == 'ls':
            for line in output:
                size, file = line.split(' ')
                pwd[file] = {} if size == 'dir' else int(size)
        else:
            print(command)
            raise ValueError()
    return hierarchy


data = parses(puzzle.input_data)

In [4]:

sample = parses("""$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
""")

In [5]:
sample

{'/': {'a': {'e': {'i': 584}, 'f': 29116, 'g': 2557, 'h.lst': 62596},
  'b.txt': 14848514,
  'c.dat': 8504156,
  'd': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}}

In [11]:
def sizes(root):
    all_sizes = []
    def _helper(node):
        size = 0
        for file, content in node.items():
            if isinstance(content, int):
                size += content
            elif isinstance(content, dict):
                size += _helper(content)
        all_sizes.append(size)
        return size
    _helper(root)
    return all_sizes

In [13]:
def solve_a(data):
    return sum(s for s in sizes(data) if s <= 100_000)

In [26]:
solve_a(sample)

95437

In [27]:
solve_a(data)

1582412

In [23]:
def solve_b(data):
    sizes_ = sizes(data)
    current_free = 70000000 - sizes_[-1]
    return min(s for s in sizes_ if (current_free + s) >= 30000000)

In [24]:
solve_b(sample)

24933642

In [25]:
solve_b(data)

3696336

In [47]:
def solve_a(data):
    def small_dirs(root):
        total_size = 0
        small_size = 0
        for file, content in root.items():
            if isinstance(content, int):
                total_size += content
            elif isinstance(content, dict):
                sub_size, sub_small = small_dirs(content)
                total_size += sub_size
                small_size += sub_small
        small_size += total_size * (total_size < 100_000)
        return total_size, small_size
    return small_dirs(data)[1]
        

95437

1582412

In [53]:
def solve_b(data):
    sizes = []
    def compute_size(root):
        total_size = 0
        for file, content in root.items():
            if isinstance(content, int):
                total_size += content
            elif isinstance(content, dict):
                total_size += compute_size(content)
        sizes.append(total_size)
        return total_size
    total_size = compute_size(data)
    current_free = 70000000 - total_size
    min_free = 30000000 - current_free
    return sorted(i for i in sizes if i >= min_free)[0]
        

24933642

3696336